In [2]:
import keras
import numpy as np
import pandas as pd
from hyperopt import STATUS_OK,Trials,fmin,hp,tpe
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import mlflow
from mlflow.models import infer_signature

In [9]:
from sklearn.datasets import load_iris
iris = load_iris()
data = pd.DataFrame(data=iris.data, columns=iris.feature_names)
data['target'] = iris.target

In [10]:
train,test=train_test_split(data,test_size=0.25,random_state=42)
train

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
4,5.0,3.6,1.4,0.2,0
32,5.2,4.1,1.5,0.1,0
142,5.8,2.7,5.1,1.9,2
85,6.0,3.4,4.5,1.6,1
86,6.7,3.1,4.7,1.5,1
...,...,...,...,...,...
71,6.1,2.8,4.0,1.3,1
106,4.9,2.5,4.5,1.7,2
14,5.8,4.0,1.2,0.2,0
92,5.8,2.6,4.0,1.2,1


In [11]:
train_x=train.drop(['target'],axis=1).values
train_y=train[['target']].values.ravel()

## test dataset
test_x=test.drop(['target'],axis=1).values
test_y=test[['target']].values.ravel()

## splitting this train data into train and validation

train_x,valid_x,train_y,valid_y=train_test_split(train_x,train_y,test_size=0.20,random_state=42)

signature=infer_signature(train_x,train_y)

In [12]:
def train_model(params,epochs,train_x,train_y,valid_x,valid_y,test_x,test_y):

    ## Define model architecture
    mean=np.mean(train_x,axis=0)
    var=np.var(train_x,axis=0)

    model=keras.Sequential(
        [
            keras.Input([train_x.shape[1]]),
            keras.layers.Normalization(mean=mean,variance=var),
            keras.layers.Dense(64,activation='relu'),
            keras.layers.Dense(1)

        ]
    )

    ## compile the model
    model.compile(optimizer=keras.optimizers.SGD(
        learning_rate=params["lr"],momentum=params["momentum"]
    ),
    loss="mean_squared_error",
    metrics=[keras.metrics.RootMeanSquaredError()]
    )

    ## Train the ANN model with lr and momentum params wwith MLFLOW tracking
    with mlflow.start_run(nested=True):
        model.fit(train_x,train_y,validation_data=(valid_x,valid_y),
                  epochs=epochs,
                  batch_size=64)
        
        ## Evaluate the model
        eval_result=model.evaluate(valid_x,valid_y,batch_size=64)

        eval_rmse=eval_result[1]

        ## Log the parameters and results
        mlflow.log_params(params)
        mlflow.log_metric("eval_rmse",eval_rmse)

        ## log the model

        mlflow.tensorflow.log_model(model,"model",signature=signature)

        return {"loss": eval_rmse, "status": STATUS_OK, "model": model}

In [13]:
def target(params):
    # MLflow will track the parameters and results for each run
    result = train_model(
        params,
        epochs=3,
        train_x=train_x,
        train_y=train_y,
        valid_x=valid_x,
        valid_y=valid_y,
        test_x=test_x,
        test_y=test_y,
    )
    return result

In [14]:
sp={
    "lr":hp.loguniform("lr",np.log(1e-5),np.log(1e-1)),
    "momentum":hp.uniform("momentum",0.0,1.0)

}

In [15]:
mlflow.set_experiment("ANNonIRIS")
with mlflow.start_run():
    # Conduct the hyperparameter search using Hyperopt
    trials=Trials()
    best=fmin(
        fn=target,
        space=sp,
        algo=tpe.suggest,
        max_evals=4,
        trials=trials
    )

    # Fetch the details of the best run
    best_run = sorted(trials.results, key=lambda x: x["loss"])[0]

    # Log the best parameters, loss, and model
    mlflow.log_params(best)
    mlflow.log_metric("eval_rmse", best_run["loss"])
    mlflow.tensorflow.log_model(best_run["model"], "model", signature=signature)

    # Print out the best parameters and corresponding loss
    print(f"Best parameters: {best}")
    print(f"Best eval rmse: {best_run['loss']}")

2024/12/28 18:08:32 INFO mlflow.tracking.fluent: Experiment with name 'ANNonIRIS' does not exist. Creating a new experiment.


Epoch 1/3                                            

1/2 ━━━━━━━━━━━━━━━━━━━━ 0s 791ms/step - loss: 1.9746 - root_mean_squared_error: 1.4052
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 249ms/step - loss: 1.8940 - root_mean_squared_error: 1.3761 - val_loss: 1.7393 - val_root_mean_squared_error: 1.3188

Epoch 2/3                                            

1/2 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 1.7516 - root_mean_squared_error: 1.3235
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step - loss: 1.7410 - root_mean_squared_error: 1.3195 - val_loss: 1.5460 - val_root_mean_squared_error: 1.2434

Epoch 3/3                                            

1/2 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 1.4056 - root_mean_squared_error: 1.1856
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step - loss: 1.4942 - root_mean_squared_error: 1.2221 - val_loss: 1.3074 - val_root_mean_squared_error: 1.1434

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 1.3074 - root_mean_squared_error: 1.1434
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 1.30